In [27]:
# necessary libraries for prediciton
import utils
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# Useful Functions

In [28]:
def pretty_matrix(matrix, row_label, col_label):
    """Pretty print of the given matrix """

    # Restraining labels that are too big
    row_label = [el[:10] + '..' if len(el) > 10 else el
                for el in row_label]
    col_label = [el[:10] + '..' if len(el) > 10 else el
                for el in col_label]

    # Stringfying everything & Joining top label
    s_matrix = [list([" "] + (col_label))] + \
               [[row_label[row_idx]] + \
                [str(e) for e in row] for row_idx, row in enumerate(matrix)]

    # Length of each matrix column
    len_s = [max(map(len, col)) for col in zip(*s_matrix)]

    # Cell formatation
    formatation = '\t'.join('{{:{}}}'.format(x) for x in len_s)

    # Apply cell formation to each matrix element
    pretty_mat = [formatation.format(*row) for row in s_matrix]

    # Print Pretty Matrix
    print('\n'.join(pretty_mat))


def display_confusion_matrix(values):
    '''Display the given array as a confusion matrix'''
    pretty_matrix([values[0:2], values[2:4]],
                  ['Actual NO', 'Actual YES'],
                  ['Predic NO', 'Predic YES'])

# Prediction Algorithms

* Decision Tree

In [29]:
def create_DT():
    '''Create a new Decision Tree'''
    # Useful DecisionTree tutorial:
    # https://www.datacamp.com/community/tutorials/decision-tree-classification-python
    return DecisionTreeClassifier()

In [30]:
def create_RF():
    '''Create a new Ranfom Forest model'''
    return RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

# Prediction

* Predictions are done in this notebook.
* It is also useful to compare how serveral algorithms perform against one another.

In [31]:
dataset =  utils.read_csv_to_df('dataset/preprocessed_data.csv')
display(dataset.head())

,date,amount,payments,account_loan_age,frequency_MI,frequency_WI,owners_count,C,monthly_loan,monthly_loan-to-monthly_receiving,monthly_only_receiving,status
0,0.000000,0.171345,0.823266,3.387097,0,1,0,0,0.823266,1.0,0.171345,-1
1,0.004721,0.301732,0.457951,4.774194,1,0,0,0,0.457951,1.0,0.301732,1
2,0.018096,0.228857,0.191996,5.483871,1,0,0,0,0.191996,1.0,0.228857,1
3,0.022817,0.188979,0.279616,5.967742,1,0,0,0,0.279616,1.0,0.188979,1
4,0.049567,0.505623,0.454642,6.580645,0,1,0,0,0.454642,1.0,0.505623,1


In [32]:
# Useful MACROS
STATUS_COL = dataset.columns.get_loc("status")
K_FOLD_NUM_SPLITS = 5
SEED = 42

In [33]:
# Setting X and Y
X = dataset.iloc[:, 0:STATUS_COL]
y = dataset.iloc[:, [STATUS_COL]]

In [34]:
# Cross validation settings
f1_scores = []
accuracies = []
confusion_matrixes = []
cv = KFold(n_splits=K_FOLD_NUM_SPLITS, random_state=SEED, shuffle=False)

# CHANGE THIS LINE TO CHANGE THE USED CLASSIFICATION METHOD
classifier = create_DT()
# classifier = create_RF()

# Applying Cross validation
for train_index, test_index in cv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Training with this fold
    classifier.fit(X_train, y_train)
    
    # Testing & Measuring accuracy
    y_pred = classifier.predict(X_test)
    f1_scores.append(metrics.f1_score(y_test, y_pred))
    accuracies.append(metrics.accuracy_score(y_test, y_pred))
    confusion_matrixes.append(metrics.confusion_matrix(y_test, y_pred).ravel())

In [35]:
# Printing the obtained results
print('Classification Method used:', classifier, '\n')
print('F1 score:', f1_scores)
print('\t > Average: ', sum(f1_scores)/len(f1_scores))
print('Accuracies: ', accuracies)
print('\t > Average:', sum(accuracies)/len(accuracies))
for cf in confusion_matrixes:
    display_confusion_matrix(cf)

('Classification Method used:', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), '\n')
('F1 score:', [0.7676767676767676, 0.7766990291262137, 0.8695652173913043, 0.9137931034482758, 0.9333333333333333])
('\t > Average: ', 0.8522134901951789)
('Accuracies: ', [0.6515151515151515, 0.6515151515151515, 0.7727272727272727, 0.8461538461538461, 0.8769230769230769])
('\t > Average:', 0.7597668997668998)
          	Predic NO	Predic YES
Actual NO 	5        	4         
Actual YES	19       	38        
          	Predic NO	Predic YES
Actual NO 	3        	9         
Actual YES	14       	40        
          	Predic NO	Predic YES
Actual NO 	1        	7         
Actual YES	8        	50        
          	

### After having our model trained we shall use the model on the data to be sumitted in the kaggle

In [36]:
test_dataset =  utils.read_csv_to_df('test_dataset/test_dataset.csv')
ids = utils.read_csv_to_df('ficheiros_competicao/loan_test.csv', delimiter=';')
display(test_dataset.head())

,date,amount,payments,account_loan_age,frequency_MI,frequency_WI,owners_count,C,monthly_loan,monthly_loan-to-monthly_receiving,monthly_only_receiving,status
0,0.000000,0.151641,0.131376,14.580645,1,0,0,0,0.131376,1.0,0.151641,NaN
1,0.001420,0.436237,0.722049,15.806452,1,0,0,0,0.722049,0.0,0.436237,NaN
2,0.007102,0.388292,0.472725,20.322581,0,0,0,0,0.472725,1.0,0.388292,NaN
3,0.025568,0.370057,0.353321,10.032258,1,0,0,0,0.353321,1.0,0.370057,NaN
4,0.025568,0.056981,0.302519,19.258065,1,0,0,0,0.302519,1.0,0.056981,NaN


In [37]:
# We now remove the Y column with NaNs
test_dataset = test_dataset.iloc[:, 0:STATUS_COL]
display(test_dataset.head())

,date,amount,payments,account_loan_age,frequency_MI,frequency_WI,owners_count,C,monthly_loan,monthly_loan-to-monthly_receiving,monthly_only_receiving
0,0.000000,0.151641,0.131376,14.580645,1,0,0,0,0.131376,1.0,0.151641
1,0.001420,0.436237,0.722049,15.806452,1,0,0,0,0.722049,0.0,0.436237
2,0.007102,0.388292,0.472725,20.322581,0,0,0,0,0.472725,1.0,0.388292
3,0.025568,0.370057,0.353321,10.032258,1,0,0,0,0.353321,1.0,0.370057
4,0.025568,0.056981,0.302519,19.258065,1,0,0,0,0.302519,1.0,0.056981


In [38]:
# Using the model to get the 'status' predictions
predictions_df = pd.DataFrame({'Id': ids['loan_id'].array,
                            'Predicted': classifier.predict(test_dataset)})

display(predictions_df)

,Id,Predicted
0,5895,1
1,7122,1
2,6173,1
3,6142,1
4,5358,1
5,6095,1
6,6878,1
7,6554,-1
8,6793,1
9,7286,1


In [39]:
# Outputting predictions to .csv
# CHANGE FILE NAME TO PRESERVE DIFFERENT INSTANCES
utils.write_df_to_csv(predictions_df, 'submission3', 'prediction.csv')